In [24]:
import numpy as np
import matplotlib.pyplot as plt

In [40]:
LOG = True

def log(*args, a=None, **kwargs):
    if LOG and (a is None or a):
        print(*args, **kwargs)

# Метод 1: "Квадратного кореня"
Використовується для симетричних матриць

In [41]:
# 1. Факторизація симетричної додатньо визначеної A = LL'

def LU_sym_factor(A):
    n = A.shape[0]
    L = np.zeros_like(A)
    for i in range(0, n):
        for j in range(0,i):
            L[i,j] = (A[i,j] - L[i,0:i] @ L[j,0:i]) / L[j,j]
        L[i,i] = np.sqrt(A[i,i] - L[i,0:i] @ L[i,0:i])
        log(L)
    return L, L.T

In [67]:
# 2. Зворотній хід. Розв'язує систему LL'x = n; (Ly=b, L'x=y)
def solve_L(L, b):
    n = L.shape[0]
    x = np.zeros(n)
    y = np.zeros(n)
    for i in range(0,n):
        y[i] = (b[i] - L[i,0:i] @ y[0:i]) / L[i,i]
        log("y:", y)
    
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - L.T[i, i:n] @ x[i:n]) / L[i,i]
        log("x:", x)
        
    return x

# Метод 2: Гауса з півотами

In [ ]:
def solve_gauss(A, b):
    n = A.shape[0]
    x_permutation = np.arange(n)
    
    ki = 0
    kj = 0
    while ki < n:
        # get the largest element of the Bottom Right (BR)
        BR = A[ki:, kj:]
        pivot = np.unravel_index(np.abs(BR).argmax(), BR.shape)
        pivot = (pivot[0] + ki, pivot[1] + kj)
        x_permutation[kj], x_permutation[pivot[1]] = pivot[1], kj
        
        
        
        
        ki+=1
        kj+=1

In [66]:
L = (np.random.rand(4,4)*100).astype(int)
for i in range(4):
    L[i, i+1:] = 0
print(L)

x = (np.random.rand(4)*100).astype(int)
b = L@L.T@x

x1 = solve_L(L, b)
print(x1)
print(x)
print(x-x1)

[[38  0  0  0]
 [66 37  0  0]
 [45 10 68  0]
 [95 82 39 33]]
y: [10826.     0.     0.     0.]
y: [10826.  6264.     0.     0.]
y: [10826.  6264.  2822.     0.]
y: [10826.  6264.  2822.  1122.]
x: [ 0.  0.  0. 34.]
x: [ 0.  0. 22. 34.]
x: [ 0. 88. 22. 34.]
x: [21. 88. 22. 34.]
[21. 88. 22. 34.]
[21 88 22 34]
[0. 0. 0. 0.]


In [63]:
b[0]/L[0,0]

12200.0

In [76]:
A = np.random.rand(4,4)*10 - 5
print(A)
np.unravel_index(np.abs(A).argmax(),A.shape)

[[ 1.57277173 -1.86313291 -4.26418127 -3.48229186]
 [-3.16225678  0.85413917  1.27784093 -1.64502252]
 [-3.54438924 -4.87123204 -0.0815651  -1.61994666]
 [ 4.08199563 -1.34305995 -3.90571449 -1.75513424]]


TypeError: _argmax_dispatcher() got an unexpected keyword argument 'keepdims'